# NLP - Homework 2
### Miguel Bonilla

1. In Python, create a method for scoring the vocabulary size of a text, and normalize the score 
from 0 to 1. It does not matter what method you use for normalization as long as you explain it 
in a short paragraph. 
Some relevant resources that you can leverage:
https://docs.tibco.com/pub/spotfire/6.5.0/doc/html/norm/norm_scale_between_0_and_1.htm
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
2. After consulting section 3.2 in chapter 1 of Bird-Klein, create a method for scoring the long-word 
vocabulary size of a text, and likewise normalize (and explain) the scoring as in step 1 above.
3. Now create a “text difficulty score” by combining the lexical diversity score from homework 1, 
and your normalized score of vocabulary size and long-word vocabulary size, in equal weighting. 
Explain what you see when this score is applied to same graded texts you used in homework 1.

In [78]:
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

- First https://www.gutenberg.org/cache/epub/14640/pg14640.txt
- Second https://www.gutenberg.org/cache/epub/14668/pg14668.txt
- Third https://www.gutenberg.org/cache/epub/14766/pg14766.txt

In [2]:
from urllib import request
url1 = "https://www.gutenberg.org/cache/epub/14640/pg14640.txt"
url2 = "https://www.gutenberg.org/cache/epub/14668/pg14668.txt"
url3 = "https://www.gutenberg.org/cache/epub/14766/pg14766.txt"

#import books
text1 = request.urlopen(url1).read().decode('utf8')
text2 = request.urlopen(url2).read().decode('utf8')
text3 = request.urlopen(url3).read().decode('utf8')

# tokenize books
text1 = word_tokenize(text1)
text2 = word_tokenize(text2)
text3 = word_tokenize(text3)

In [3]:
## text 1 find beginning and end
print('beginning of text 1 is: ',text1.index('LESSON')) #beginning
print('end of text 1 is: ',text1.index('PHONIC')-1) #end
## text 2 find beginning and end
print('beginning of text 2 is: ',text2.index('LESSON')) #beginning
print('end of text 2 is: ',text2.index('Tennyson')) #end
## text 3 find beginning and end
print('beginning of text 3 is: ',text3.index('SHEPHERD')-4) #beginning
print('end of text 3 is: ',text3.index('Follen')) #end

beginning of text 1 is:  1125
end of text 1 is:  9335
beginning of text 2 is:  1500
end of text 2 is:  22004
beginning of text 3 is:  2116
end of text 3 is:  34675


In [4]:
text1 = text1[1125:9335]
text2 = text2[1500:22004]
text3 = text3[2116:34675]

## Normalized Vocabulary Scoring

In [44]:
texts = [text1,text2,text3]
    

In [139]:
scores = []
sizes = []
def vocab_score(text_list):
    for i in range(len(text_list)):
        vocab_size = len(set(text_list[i]))
        sizes.append(vocab_size)
    for i in range(len(sizes)):
        norm = (sizes[i]-np.asarray(sizes).min())/(np.asarray(sizes).max()-np.asarray(sizes).min())
        scores.append(norm)

In [140]:
vocab_score(texts)
scores

[0.0, 0.7011711371363808, 1.0]

In [141]:
from sklearn.preprocessing import MinMaxScaler


In [153]:
min_max = MinMaxScaler()
data = [[-1],[6],[10],[18]]

In [158]:
data = np.asarray(sizes).reshape(-1,1)

In [159]:
min_max.fit_transform(data)

array([[0.        ],
       [0.70117114],
       [1.        ]])